In [53]:
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import imageio

In [54]:
from scipy.optimize import linprog


In [55]:
dataset = "001"
# dataset = "002"
# dataset = "003"
# dataset = "004"
# dataset = "005"
# dataset = "006"
# dataset = "007"
# dataset = "008"
# dataset = "009"
# dataset = "010"

In [56]:
detfile = "data_detection/" + dataset + "/detection.txt"
imgdir  = "data_detection/" + dataset + "/images/"

In [57]:
file = open(detfile, "r")
l0 = file.readline()
l1 = file.readline()
f,cx,cy = np.array(l1.split()).astype(int)
file.close()

In [58]:
detections = pd.read_csv(detfile,delimiter=" ",skiprows=2)
images = np.unique(detections["#image"].values) #keep image indices because there are numbers missing between 4 and 167

In [59]:
images

array([  4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        71,  72,  74,  75,  76,  77,  78,  79,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167], dtype=int64)

In [60]:
images.size

157

In [61]:
counter=[]

In [62]:
for i in images:
    df=detections[detections['#image']==i]
    n=df.shape[0]
    counter.append(n)

In [63]:
counter

[5,
 5,
 7,
 4,
 5,
 5,
 3,
 5,
 6,
 6,
 5,
 4,
 4,
 6,
 5,
 8,
 5,
 5,
 7,
 5,
 3,
 5,
 5,
 5,
 3,
 5,
 4,
 3,
 1,
 1,
 2,
 3,
 2,
 4,
 7,
 13,
 10,
 4,
 13,
 6,
 4,
 11,
 6,
 7,
 4,
 2,
 3,
 1,
 5,
 5,
 3,
 4,
 2,
 2,
 8,
 5,
 5,
 9,
 2,
 3,
 2,
 5,
 7,
 8,
 2,
 3,
 7,
 3,
 4,
 1,
 1,
 3,
 1,
 3,
 3,
 6,
 2,
 2,
 1,
 5,
 1,
 2,
 3,
 3,
 2,
 5,
 7,
 4,
 5,
 5,
 4,
 7,
 6,
 3,
 4,
 9,
 3,
 10,
 11,
 1,
 5,
 5,
 3,
 9,
 8,
 12,
 9,
 10,
 10,
 14,
 15,
 21,
 18,
 14,
 15,
 10,
 13,
 11,
 12,
 16,
 13,
 6,
 7,
 9,
 8,
 16,
 15,
 6,
 13,
 12,
 5,
 4,
 5,
 3,
 9,
 9,
 10,
 3,
 15,
 1,
 7,
 8,
 3,
 3,
 6,
 2,
 6,
 3,
 16,
 1,
 3,
 10,
 6,
 1,
 6,
 10,
 7]

In [64]:
M=max(counter)
M #the maximum number of points in an image

21

In [91]:
def distance(u,v):
    return(np.linalg.norm(u[1:2]-v[1:2]) + np.linalg.norm(u[-1]-v[-1]))

In [92]:
method = 'revised simplex'

def computeTransport(X, Y):

    nbPtsNuage1 = X.shape[0]
    nbPtsNuage2 = Y.shape[0]
    C = np.zeros([nbPtsNuage1, nbPtsNuage2])

    for i in range(nbPtsNuage1):
        for j in range(nbPtsNuage2):
            C[i,j] = distance(X[i],Y[j])

    c = np.ravel(C)
    nbVariables = nbPtsNuage1*nbPtsNuage2
    nbContraintes = nbPtsNuage2 + nbPtsNuage1

    ##Cas 1 : autant de points dans les 2 nuages

    if nbPtsNuage1 == nbPtsNuage2:
        b_eq = np.ones(nbContraintes)

        A_eq = np.zeros([nbContraintes,nbVariables])
        #les premières lignes correspondent aux contraintes de sortie. c'est à dire pour
        #chaque point du Nuage1, il faut le connecter à un seul point du nuage 2.
        for i in range(nbPtsNuage1):
            for j in range(nbPtsNuage2):
                A_eq[i,i*nbPtsNuage2 +j] = 1
        #les dernieres lignes correspondent aux contraintes d'entrée. Pour chaque point
        #dans le nuage d'arrivée, il peut recevoir 1 seul nuage de sortie.
        for i in range(nbPtsNuage2):
            for j in range(nbPtsNuage1):
                A_eq[i + nbPtsNuage1, i + j*nbPtsNuage2] = 1

        P = linprog(c, A_eq = A_eq, b_eq = b_eq, bounds=(0,1), method=method)

    ##Cas 2 : nbPtsNuage1 > nbPtsNuage2

    elif nbPtsNuage1 > nbPtsNuage2:
        b_eq = np.zeros(nbContraintes)
        b_eq[nbPtsNuage1:] = np.ones(nbPtsNuage2)

        b_ub = np.zeros(nbContraintes)
        b_ub[:nbPtsNuage1] = np.ones(nbPtsNuage1)

        A_eq = np.zeros([nbContraintes,nbVariables])
        #les premieres lignes sont nulles ; les contraintes d'égalité sont :
        #dans le nuage d'arrivée, un point doit recevoir pile 1 point de sortie.
        for i in range(nbPtsNuage2):
            for j in range(nbPtsNuage1):
                A_eq[i + nbPtsNuage1, i + j*nbPtsNuage2] = 1

        A_ub = np.zeros([nbContraintes,nbVariables])
        #les contraintes d'inégalité sont les contrainte de sortie, cad pour
        #chaque point du Nuage1, il faut le connecter à au plus 1 point du nuage 2.
        for i in range(nbPtsNuage1):
            for j in range(nbPtsNuage2):
                A_ub[i,i*nbPtsNuage2 +j] = 1

        P = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds=(0,1), method=method)

    ##Cas 3 : nbPtsNuage1 < nbPtsNuage2

    else:
        b_eq = np.zeros(nbContraintes)
        b_eq[:nbPtsNuage1] = np.ones(nbPtsNuage1)

        b_ub = np.zeros(nbContraintes)
        b_ub[nbPtsNuage1:] = np.ones(nbPtsNuage2)

        A_eq = np.zeros([nbContraintes,nbVariables])
        #les dernières lignes sont nulles ; les contraintes d'égalité sont :
        #chaque point du nuage de départ doit être connecté à exactement 1 point
        #du nuage d'arrivée.
        for i in range(nbPtsNuage1):
            for j in range(nbPtsNuage2):
                A_eq[i,i*nbPtsNuage2 +j] = 1

        A_ub = np.zeros([nbContraintes,nbVariables])
        #les contraintes d'inégalité sont les contrainte d'entrée, cad que
        #chaque point du Nuage2 ne peut recevoir qu'au plus 1 point du nuage 2.
        for i in range(nbPtsNuage2):
            for j in range(nbPtsNuage1):
                A_ub[i + nbPtsNuage1, i + j*nbPtsNuage2] = 1

        P = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds=(0,1), method=method)

    ##Résultat : la matrice de connexion x

    #on 'unravel' le résultat P.x, à la main :
    x = np.array([P.x[k*nbPtsNuage2:(k+1)*nbPtsNuage2] for k in range(nbPtsNuage1)])
    #x est la matrice de couplage, cad la matrice des pij
    return(x)

In [93]:
IndexImage1=(detections["#image"]==images[0])
X_im1 =  detections[IndexImage1][:].values #dataframe corresponding to image 1
#trajet1=X_im1[:,1:3]   #points in first image initialize trajectory; Coordinates of a point in image1=A line in trajet
trajet1=X_im1
m=len(trajet1)

In [94]:
trajet1

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1]], dtype=int64)

In [95]:
for i in range(m,M+1):
    trajet1=np.append(trajet1,[[4,0,0,0,0]],0)
    

In [159]:
trajet1

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0]], dtype=int64)

In [160]:
d=np.array([traj[0:5] for traj in trajet1])

In [161]:
len(d)

22

In [162]:
ttt=[]

In [163]:
d

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0]], dtype=int64)

In [164]:
d.tolist()

[[4, 151, -23, 179, 1],
 [4, 153, 25, 174, 2],
 [4, 158, 92, 174, 1],
 [4, 155, 86, 160, 1],
 [4, 158, 94, 174, 1],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0],
 [4, 0, 0, 0, 0]]

In [165]:
len(d)

22

In [166]:
for i in range(len(d)):
    if d[i,-1] !=0:
        ttt.append(d[i,0:5])

In [167]:
ttt=np.array(ttt)

In [168]:
ttt

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1]], dtype=int64)

In [177]:
# first function of trajectories when P is constructed only with equalities
def trajectory(trajet,Y):
    #from trajet until now, we will construct the X to put in f=computeTransport, 
    #otherwise, if we let f treat initial dataframes it will not take into consideration the rearrangements done in previous steps while construction trajectories
    d=np.array([traj[0:5] for traj in trajet1])
    d.tolist()
    ttt=[]
    for i in range(len(d)):
        if d[i,-1] !=0:
            ttt.append(d[i,0:5])
    X=np.array(ttt)       
    P=computeTransport(X, Y)
    nbPtsX = X.shape[0]
    nbPtsY = Y.shape[0]
    tr=np.array(trajet)
    newV=np.zeros((nbPtsX,5))
    #dim=tr.shape #dimensions of trajet i.e if there are m points in an image, dim=(m,2)! m=nbPtsX
    if nbPtsX==nbPtsY:        
        for i in range(nbPtsX):
            for j in range(nbPtsY):
                if P[i,j]==1:
                    newV[i,:]=Y[j,0:5]          
    elif nbPtsX > nbPtsY:
        for i in range(nbPtsX):
            for j in range(nbPtsY):
                if P[i,j]==1:
                    newV[i,:]=Y[j,0:5]
            if (all(P[i,j] == 0 for j in range(nbPtsY))):  
                newV[i,:]=X[i,0:5]
    else:
        Matchedpoints=[]
        k=0
        for i in range(nbPtsX):
            for j in range(nbPtsY):
                if P[i,j]==1:
                    newV[i,:]=Y[j,0:5]
                    k=j
                    Matchedpoints.append(k)
        for j in range(nbPtsY):
            if j not in Matchedpoints:
                newV=np.append(newV,Y[j,0:5])
    m=len(newV)
    for i in range(m,M+1):
        newV=np.append(newV,[[0,0,0,0,0]],0)
    res=np.append(tr,newV,1)    
    trajet=res.tolist()
    return(trajet) 
#initially trajet has one column. Each line is a point. After one iteration trajet has two columns. Point trajet[m,n] goes to trajet[m,n+1] in the trajectory path 

In [178]:
trajet1

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0],
       [  4,   0,   0,   0,   0]], dtype=int64)

In [179]:
#first 
maskX=(detections["#image"]==4)
maskY = (detections["#image"]==5)
X1 =  detections[maskX][:].values #dataframe corresponding to image 4
Y1 = detections[maskY][:].values #dataframe corresponding to image 5
trajet=trajectory(trajet1,Y1)

<ipython-input-92-7cf3c4bc9e1d>:34: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  P = linprog(c, A_eq = A_eq, b_eq = b_eq, bounds=(0,1), method=method)


In [180]:
trajet

[[4.0, 151.0, -23.0, 179.0, 1.0, 5.0, 154.0, -26.0, 183.0, 1.0],
 [4.0, 153.0, 25.0, 174.0, 2.0, 5.0, 153.0, 25.0, 174.0, 2.0],
 [4.0, 158.0, 92.0, 174.0, 1.0, 5.0, 158.0, 94.0, 174.0, 1.0],
 [4.0, 155.0, 86.0, 160.0, 1.0, 5.0, 155.0, 85.0, 160.0, 1.0],
 [4.0, 158.0, 94.0, 174.0, 1.0, 5.0, 158.0, 92.0, 174.0, 1.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [181]:
Y1

array([[  5, 154, -26, 183,   1],
       [  5, 153,  25, 174,   2],
       [  5, 158,  92, 174,   1],
       [  5, 155,  85, 160,   1],
       [  5, 158,  94, 174,   1]], dtype=int64)

In [182]:
X1

array([[  4, 151, -23, 179,   1],
       [  4, 153,  25, 174,   2],
       [  4, 158,  92, 174,   1],
       [  4, 155,  86, 160,   1],
       [  4, 158,  94, 174,   1]], dtype=int64)

In [183]:
P=computeTransport(X1,Y1)

<ipython-input-92-7cf3c4bc9e1d>:34: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  P = linprog(c, A_eq = A_eq, b_eq = b_eq, bounds=(0,1), method=method)


In [184]:
P

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])

In [190]:
maskX=(detections["#image"]==5) #cause we want to start with image 5
maskY = (detections["#image"]==7)
X =  detections[maskX][:].values #dataframe corresponding to image i
Y = detections[maskY][:].values #dataframe corresponding to image i+1
trajet=trajectory(trajet,Y)

In [191]:
trajet

[[4.0,
  151.0,
  -23.0,
  179.0,
  1.0,
  5.0,
  154.0,
  -26.0,
  183.0,
  1.0,
  4.0,
  151.0,
  -23.0,
  179.0,
  1.0,
  4.0,
  151.0,
  -23.0,
  179.0,
  1.0,
  4.0,
  151.0,
  -23.0,
  179.0,
  1.0],
 [4.0,
  153.0,
  25.0,
  174.0,
  2.0,
  5.0,
  153.0,
  25.0,
  174.0,
  2.0,
  7.0,
  153.0,
  26.0,
  174.0,
  1.0,
  7.0,
  153.0,
  26.0,
  174.0,
  1.0,
  7.0,
  153.0,
  26.0,
  174.0,
  1.0],
 [4.0,
  158.0,
  92.0,
  174.0,
  1.0,
  5.0,
  158.0,
  94.0,
  174.0,
  1.0,
  7.0,
  158.0,
  94.0,
  174.0,
  1.0,
  7.0,
  158.0,
  94.0,
  174.0,
  1.0,
  7.0,
  158.0,
  94.0,
  174.0,
  1.0],
 [4.0,
  155.0,
  86.0,
  160.0,
  1.0,
  5.0,
  155.0,
  85.0,
  160.0,
  1.0,
  7.0,
  155.0,
  86.0,
  160.0,
  1.0,
  7.0,
  155.0,
  86.0,
  160.0,
  1.0,
  7.0,
  155.0,
  86.0,
  160.0,
  1.0],
 [4.0,
  158.0,
  94.0,
  174.0,
  1.0,
  5.0,
  158.0,
  92.0,
  174.0,
  1.0,
  7.0,
  158.0,
  92.0,
  174.0,
  1.0,
  7.0,
  158.0,
  92.0,
  174.0,
  1.0,
  7.0,
  158.0,
  92.0,
  174.0

In [108]:
X

array([[  5, 154, -26, 183,   1],
       [  5, 153,  25, 174,   2],
       [  5, 158,  92, 174,   1],
       [  5, 155,  85, 160,   1],
       [  5, 158,  94, 174,   1]], dtype=int64)

In [109]:
Y

array([[  7, 153,  26, 174,   1],
       [  7, 158,  92, 174,   1],
       [  7, 155,  86, 160,   1],
       [  7, 158,  94, 174,   1]], dtype=int64)

In [110]:
P=computeTransport(X, Y)

In [111]:
P

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [61]:
for i in range(5):
    if (all(P[i,j] == 0 for j in range(4))):
        print(i)

0


In [114]:
nbPtsX = X.shape[0]
nbPtsY = Y.shape[0]
newV=np.zeros((nbPtsX,2))


In [117]:
for i in range(nbPtsX):
    for j in range(nbPtsY):
        if P[i,j]==1:
            newV[i,:]=Y[j,1:3]
    if (all(P[i,j] == 0 for j in range(nbPtsY))):  
                newV[i,:]=X[i,1:3]

In [118]:
newV

array([[154., -26.],
       [153.,  26.],
       [158.,  94.],
       [155.,  86.],
       [158.,  92.]])